
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a0/HDF_logo.svg" align="right" width="20%">

## Julia HDF5  and JLD
[Lázaro Alonso](https://lazarusa.github.io/Webpage/index.html)
___

Hierarchical Data Format (HDF) is a set of file formats (HDF4, HDF5) designed to store and organize large amounts of data. Originally developed at the National Center for Supercomputing Applications, it is supported by The HDF Group, a non-profit corporation whose mission is to ensure continued development of HDF5 technologies and the continued accessibility of data stored in HDF.

- https://en.wikipedia.org/wiki/Hierarchical_Data_Format

# Outline
* [HDF5](#HDF5)
	* [Write and read file](#Blocked-Algorithms)
	* [Open file](#Exercise:--Compute-the-mean-using-a-blocked-algorithm)
		* [Read complete file](#Exercise:--Compute-the-mean)
		* [Read slices](#Example)
	* [Storing big arrays incrementally](#Exercise:--Meteorological-data)
		* [Open-close method](#Exercise:--Subsample-and-store)
        * [Open-do-file method](#Exercise:--Subsample-and-store)
* [JLD](#Example:-Lennard-Jones-potential)
	* [save](#Dask-version)
	* [open](#Profiling)
    * [read](#Exercise:--Subsample-and-store)


> HDF5 stands for Hierarchical Data Format v5 and is closely modeled on file systems. In HDF5, a "group" is analogous to a directory, a "dataset" is like a file. HDF5 also uses "attributes" to associate metadata with a particular group or dataset. 
- https://github.com/JuliaIO/HDF5.jl
- https://github.com/JuliaIO/HDF5.jl/blob/master/doc/hdf5.md
- https://github.com/JuliaIO/JLD.jl

In [42]:
using HDF5, JLD, Test, ProgressMeter, Pkg

In [45]:
include("pkgversions.jl");

In [46]:
pkgsVersion()

Julia == 1.0.0
HDF5 == 0.10.3
IJulia == 1.14.1
JLD == 0.9.1
ProgressMeter == 0.8.0


### Simple writting and loading... 
Suppose that you have two arrays, and want to saved into the same file

In [2]:
arr1 = rand(1000,1000)
arr2 = rand(10,25)

10×25 Array{Float64,2}:
 0.0585158  0.466921   0.770585  0.500985   …  0.944522   0.405942  0.624675 
 0.317547   0.99584    0.819705  0.711076      0.0566668  0.481621  0.184016 
 0.04748    0.913573   0.577739  0.250829      0.0866955  0.813097  0.78869  
 0.069968   0.159338   0.163661  0.286025      0.948984   0.880552  0.5256   
 0.525331   0.0139734  0.602305  0.0195411     0.0854151  0.477742  0.737524 
 0.038006   0.907323   0.821577  0.125066   …  0.713056   0.703199  0.0031019
 0.0635439  0.633875   0.153423  0.392468      0.211248   0.769062  0.601285 
 0.910495   0.88182    0.964707  0.71539       0.606333   0.838189  0.174696 
 0.658424   0.0541998  0.33252   0.109155      0.139272   0.266625  0.155609 
 0.380098   0.948958   0.235199  0.534457      0.131102   0.707169  0.993895 

### HDF5

One way to save to a `h5` file is to type `h5write("filename.h5", "name", arr1)`, where `"name"`(creates a dataset) is a `key` for the array `arr1` to be saved. 

In [3]:
h5write("testfile.h5", "arr1", arr1) 
# @write "testfile.h5" arr1 
# h5write("testfile.h5", "arr1", arr1, "arr2", arr2) for several datasets at the same time 
# the first option is more specific (you can tell the name of the dataset to be saved), so I liked more. 

Now, to read (load) this file we simply type: 

In [4]:
arr_load = h5read("testfile.h5", "arr1")

1000×1000 Array{Float64,2}:
 0.462136   0.363221   0.586819    …  0.215662   0.924883   0.10745  
 0.716234   0.850832   0.372387       0.828721   0.300957   0.705052 
 0.390847   0.942859   0.304519       0.881418   0.577672   0.466735 
 0.61313    0.581249   0.285395       0.976312   0.274994   0.0771957
 0.813772   0.358512   0.916714       0.881597   0.53557    0.190741 
 0.754272   0.332333   0.765953    …  0.558702   0.43669    0.506043 
 0.828676   0.168348   0.214055       0.683629   0.44977    0.818357 
 0.0909521  0.851579   0.0993371      0.427194   0.191073   0.926275 
 0.129248   0.101001   0.202388       0.05458    0.874114   0.587094 
 0.298607   0.243584   0.00180247     0.421136   0.933622   0.68531  
 0.97151    0.800618   0.773192    …  0.520547   0.907438   0.959663 
 0.284999   0.932764   0.613047       0.213291   0.352155   0.835357 
 0.367318   0.786535   0.234611       0.943516   0.861677   0.462516 
 ⋮                                 ⋱                          

Please note that we need to call also the dataset of interest, `arr1`. And another way will be to `h5open` the file and then `read, dump` its content. 

In [5]:
farr = h5open("testfile.h5", "r")

HDF5 data file: testfile.h5

In [6]:
dump(farr)

HDF5File
  id: Int64 72057594037927938
  filename: String "testfile.h5"


If we don't know the datasets stored in the `h5` file, then we could use `names`, 

In [7]:
names(farr)

1-element Array{String,1}:
 "arr1"

In [8]:
readmmap(farr["arr1"])

1000×1000 Array{Float64,2}:
 0.462136   0.363221   0.586819    …  0.215662   0.924883   0.10745  
 0.716234   0.850832   0.372387       0.828721   0.300957   0.705052 
 0.390847   0.942859   0.304519       0.881418   0.577672   0.466735 
 0.61313    0.581249   0.285395       0.976312   0.274994   0.0771957
 0.813772   0.358512   0.916714       0.881597   0.53557    0.190741 
 0.754272   0.332333   0.765953    …  0.558702   0.43669    0.506043 
 0.828676   0.168348   0.214055       0.683629   0.44977    0.818357 
 0.0909521  0.851579   0.0993371      0.427194   0.191073   0.926275 
 0.129248   0.101001   0.202388       0.05458    0.874114   0.587094 
 0.298607   0.243584   0.00180247     0.421136   0.933622   0.68531  
 0.97151    0.800618   0.773192    …  0.520547   0.907438   0.959663 
 0.284999   0.932764   0.613047       0.213291   0.352155   0.835357 
 0.367318   0.786535   0.234611       0.943516   0.861677   0.462516 
 ⋮                                 ⋱                          

In [9]:
close(farr)

Let's check how long does it take to read this files

In [10]:
@time arr_load = h5read("testfile.h5", "arr1");

  0.008087 seconds (32 allocations: 7.631 MiB)


It's also posible to load just a small slice from the file _(and dataset)_, useful when memory is an issue. 

In [12]:
@time data = h5read("testfile.h5", "arr1", (2:15, 2:15))

  0.001026 seconds (51 allocations: 4.063 KiB)


14×14 Array{Float64,2}:
 0.850832  0.372387    0.120355   …  0.94178   0.194467  0.870404  
 0.942859  0.304519    0.664439      0.229889  0.289218  0.716554  
 0.581249  0.285395    0.195234      0.606101  0.684714  0.00566195
 0.358512  0.916714    0.356562      0.319648  0.625603  0.07534   
 0.332333  0.765953    0.366279      0.452046  0.691756  0.366683  
 0.168348  0.214055    0.820479   …  0.801266  0.811401  0.493455  
 0.851579  0.0993371   0.866315      0.953533  0.421952  0.484452  
 0.101001  0.202388    0.345535      0.367557  0.98411   0.828695  
 0.243584  0.00180247  0.669277      0.789936  0.699234  0.498496  
 0.800618  0.773192    0.216155      0.147724  0.8814    0.719916  
 0.932764  0.613047    0.0584236  …  0.150815  0.517029  0.38775   
 0.786535  0.234611    0.959086      0.171457  0.216337  0.0448354 
 0.657302  0.675079    0.313684      0.290919  0.600572  0.312164  
 0.252983  0.933837    0.971975      0.119775  0.363436  0.401983  

### Saving a new dataset in the same file is really easy, 

In [13]:
h5write("testfile.h5", "arr2", arr2) 

In [14]:
farr2 = h5open("testfile.h5", "r")

HDF5 data file: testfile.h5

In [15]:
names(farr2)

2-element Array{String,1}:
 "arr1"
 "arr2"

In [16]:
read(farr2["arr2"])

10×25 Array{Float64,2}:
 0.0585158  0.466921   0.770585  0.500985   …  0.944522   0.405942  0.624675 
 0.317547   0.99584    0.819705  0.711076      0.0566668  0.481621  0.184016 
 0.04748    0.913573   0.577739  0.250829      0.0866955  0.813097  0.78869  
 0.069968   0.159338   0.163661  0.286025      0.948984   0.880552  0.5256   
 0.525331   0.0139734  0.602305  0.0195411     0.0854151  0.477742  0.737524 
 0.038006   0.907323   0.821577  0.125066   …  0.713056   0.703199  0.0031019
 0.0635439  0.633875   0.153423  0.392468      0.211248   0.769062  0.601285 
 0.910495   0.88182    0.964707  0.71539       0.606333   0.838189  0.174696 
 0.658424   0.0541998  0.33252   0.109155      0.139272   0.266625  0.155609 
 0.380098   0.948958   0.235199  0.534457      0.131102   0.707169  0.993895 

In [17]:
close(farr2)

### Storing big arrays incrementally... 
This is useful to incrementally save to very large datasets you don't want to keep in memory.

Here, first let's define a function to get the file size of our files, as follows: 

In [18]:
function szprint(szfile)
    if szfile <= 10^6
        println("File size = $(szfile/10^3) KB")
    elseif 10^6 < szfile <= 10^9
        println("File size = $(szfile/10^6) MB")
    elseif 10^9 < szfile <= 10^12
        println("File size = $(szfile/10^9) GB")
    elseif 10^12 < szfile <= 10^15
        println("File size = $(szfile/10^12) TB")
    else
        println("File size is really big. Petabytes or more!")
    end
end

szprint (generic function with 1 method)

In [19]:
szprint(700001000)

File size = 700.001 MB


> The argument is `szfile = filesize("filename.h5")`, which gives you a `byte` count. 

**Here, I will show two ways to do it.**

#### First one 

In [20]:
f = h5open("incrementalFilev1.h5", "w")
dset_A = d_create(f,"arr1",datatype(Float64), dataspace(10^6,1))
for i in 1:10^4:10^6
    dset_A[i:i + 10^4 - 1,1] = rand(10^4)
    szfile = filesize("incrementalFilev1.h5")
    szprint(szfile)
    sleep(.1)
    IJulia.clear_output(true)
end
flush(f)
close(f)

File size = 8.002048 MB


Using this method you always need to `close` the file. 

In [21]:
function szprintp(szfile)
    if szfile <= 10^6
        return (szfile/10^3, "KB") 
    elseif 10^6 < szfile <= 10^9
        return (szfile/10^6, "MB")
    elseif 10^9 < szfile <= 10^12
        return (szfile/10^9, "GB")
    elseif 10^12 < szfile <= 10^15
        return(szfile/10^12, "TB")
    else
        return ("=> Petabytes")
    end
end

szprintp (generic function with 1 method)

In [22]:
szprintp(1000)

(1.0, "KB")

#### Second option 

In [23]:
h5open("incrementalFilev2.h5", "w") do file
    dset = d_create(file, "arr1k", datatype(Float64), dataspace(10^6,1))
    p = Progress(100, dt=0.1, desc="Saving...", color=:blue)
    for iter in 1:10^4:10^6
        dset[iter:iter + 10^4 - 1,1] = rand(10^4)
        szfile = filesize("incrementalFilev2.h5")        
        ProgressMeter.next!(p; showvalues = [(:Filesize, szprintp(szfile))])
        sleep(0.1)
        IJulia.clear_output(true)
    end
    end;


Saving...100%|██████████████████████████████████████████| Time: 0:00:11
  Filesize:  (8.002048, "MB")


and, using the `do` method there is no need for closing. 

### Now, let's read  some of  files 

In [24]:
f1 = h5open("incrementalFilev1.h5","r")

HDF5 data file: incrementalFilev1.h5

In [25]:
names(f1)

1-element Array{String,1}:
 "arr1"

In [26]:
A_mmaped = readmmap(f1["arr1"])

1000000×1 Array{Float64,2}:
 0.5894602638501172 
 0.4184764328371329 
 0.439757133232755  
 0.48351324327234546
 0.0803237890638866 
 0.2800074718410055 
 0.29435621582954297
 0.7790651181756039 
 0.5452231923974442 
 0.3201954906219213 
 0.6008114467899888 
 0.2227568611082542 
 0.4012946570840701 
 ⋮                  
 0.2800807684963549 
 0.03433193720859573
 0.6105182059639942 
 0.7690635630025138 
 0.08902317967935702
 0.5829474472490959 
 0.07238830013505937
 0.769639914822033  
 0.02367084838587652
 0.6953015780950993 
 0.449456799661053  
 0.9786655141740772 

In my daily workflow I usually need to save matrices, a lot! So, this is how I do it, _(additional options, like chuncks and compress are also available)..._ 

Suppose I want to save $100$ realizations where the output in each case is a $100\times100$ matrix, then 

In [27]:
#rand(100,100)

In [39]:
fm = h5open("MatrixFile.h5", "w")
dset_fm = d_create(fm,"arraym",datatype(Float64), dataspace(100,100,100)) #, "chunk", (100,100,1))
p = Progress(100, dt=0.1, desc="Saving...", color=:blue)
for i in 1:100
    dset_fm[:, :, i] = rand(100,100)
    szfilem = filesize("MatrixFile.h5")
    ProgressMeter.next!(p; showvalues = [(:File_Size, szprintp(szfilem))])
    sleep(0.1)
    IJulia.clear_output(true)
end
flush(fm)
close(fm)


Saving...100%|██████████████████████████████████████████| Time: 0:00:11
  File_Size:  (8.002048, "MB")


Sometimes `chunking` improves efficiency if you write or extract small segments or slices from an array. 

In [29]:
fmatrix = h5open("MatrixFile.h5", "r")

HDF5 data file: MatrixFile.h5

In [30]:
names(fmatrix)

1-element Array{String,1}:
 "arraym"

In [31]:
close(fmatrix)

In [32]:
h5read("MatrixFile.h5", "arraym", (:, :, 1))

100×100×1 Array{Float64,3}:
[:, :, 1] =
 0.928248  0.0696589  0.931383   …  0.904124   0.843308    0.914293 
 0.108408  0.964235   0.948981      0.900878   0.00742996  0.355971 
 0.426537  0.393569   0.852943      0.401384   0.411922    0.724338 
 0.171431  0.743979   0.939311      0.941813   0.889796    0.0774215
 0.91756   0.843584   0.222655      0.24704    0.730844    0.397121 
 0.3682    0.492465   0.980167   …  0.407092   0.61815     0.724063 
 0.773456  0.572464   0.0536272     0.740469   0.377556    0.0030069
 0.906614  0.047466   0.772832      0.875563   0.268535    0.645895 
 0.539327  0.169885   0.777984      0.0979376  0.106558    0.502408 
 0.754408  0.326626   0.897094      0.406862   0.114632    0.712558 
 0.979511  0.63735    0.3329     …  0.569844   0.684612    0.923897 
 0.201261  0.538277   0.58591       0.430929   0.019732    0.784963 
 0.2028    0.179967   0.300326      0.199255   0.0817853   0.0937971
 ⋮                               ⋱                             

## JLD 

In [33]:
save("testfile.jld", "arr1", arr1)
#@save "testfile.jld" arr1
# again, I prefer the first option, just for control. 

In [34]:
openfile = jldopen("testfile.jld","r", mmaparrays=true)
#load("testfile.jld", "arr1")
#@load "testfile.jld" arr1
# eval(:arr1)
# these last options are also valid, but for big files the first option is better. 

Julia data file version 0.1.2: testfile.jld

Once the `jld` file is open it is also posible to access small slices, to avoid  memory issues. 

In [35]:
@time read(openfile["arr1"])[1:15, 2:5]

  0.144443 seconds (205.12 k allocations: 10.180 MiB, 10.17% gc time)


15×4 Array{Float64,2}:
 0.363221  0.586819    0.84582    0.0462436
 0.850832  0.372387    0.120355   0.0970798
 0.942859  0.304519    0.664439   0.120625 
 0.581249  0.285395    0.195234   0.61301  
 0.358512  0.916714    0.356562   0.388615 
 0.332333  0.765953    0.366279   0.305589 
 0.168348  0.214055    0.820479   0.74564  
 0.851579  0.0993371   0.866315   0.939746 
 0.101001  0.202388    0.345535   0.142217 
 0.243584  0.00180247  0.669277   0.837023 
 0.800618  0.773192    0.216155   0.0456821
 0.932764  0.613047    0.0584236  0.557297 
 0.786535  0.234611    0.959086   0.474546 
 0.657302  0.675079    0.313684   0.802757 
 0.252983  0.933837    0.971975   0.569784 

In [36]:
@load "testfile.jld"

1-element Array{Symbol,1}:
 :arr1

As before, we can inspect the names(datasets) stored in this file as 

In [37]:
names(openfile)

1-element Array{String,1}:
 "arr1"

In [38]:
close(openfile)

- https://discourse.julialang.org/t/how-to-read-range-of-jld-file/954/6
- https://github.com/timholy/ProgressMeter.jl

### Storing big arrays incrementally... JLD version, waiting for `d_create` for JLD. Currenly just `g_create` is available. 